In this .ipynb, I want to explore datasets that really can reflect topic coverage importance in downstream retrieval performance

In [3]:
import ir_datasets
print("loading car/v1.5/train/fold0 dataset...")
dataset = ir_datasets.load("car/v1.5/train/fold0")

print("dataset loaded, turning into a list of documents...")
# doc_list = list(dataset.docs_iter())

# print("dataset has", len(doc_list), "documents")
i = 0
for d in dataset.docs_iter():
    print(d.doc_id, d.text)
    i += 1
    if i > 10:
        break
    


loading car/v1.5/train/fold0 dataset...
dataset loaded, turning into a list of documents...
0000000e7e72cafb61a9f356b7dceb25c5e028db Ukraine was one of the most dangerous places for journalists in the world during the euromaidan demonstrations and the war in Donbass. A May 2014 report from the OSCE found approximately 300 instances of perceived violent attacks on the media in Ukraine since November 2013.  78 journalists were abducted and illegally detained by various groups in 2014 - a new category of professional risk; 20 such cases happened in Donetsk in April 2014. In July 2014 a firebomb was thrown at the TV channel ''112 Ukraine''.
0000005eafb11cbd7e9780161f9f0a0a6fd897cd '''October 4'''  Missouri opened its season with a 3-0 win at Chicago, the Maroons' only loss of the season.  Notre Dame opened its season with a 40-0 win over Lombard College.  Stanford beat Occidental College 20-6, and California beat St. Mary's 17-7.  Army beat St. Louis 17-0,  Yale beat North Carolina 27-0, a

In [ ]:
import ir_datasets
print("loading car/v1.5/train/fold0 dataset...")
dataset = ir_datasets.load("car/v1.5/train/fold0")

i = 0
for query in dataset.queries_iter():
    print(f"query id: {query.query_id}, query text: {query.text}")
    i += 1
    if i > 10:
        break

i = 0
for qrel in dataset.qrels_iter():
    print(f"qrel query id: {qrel.query_id}, doc id: {qrel.doc_id}, relevance: {qrel.relevance}")
    i += 1
    if i > 10:
        break
for doc in dataset.docs_iter():
    print(f"doc id: {doc.doc_id}, text: {doc.text}")
    i += 1
    if i > 10:
        break


query id: Kindertotenlieder/Text%20and%20music, query text: Kindertotenlieder Text and music
query id: Kindertotenlieder/Composition%20and%20premiere, query text: Kindertotenlieder Composition and premiere
query id: Kindertotenlieder/Scoring%20and%20performance, query text: Kindertotenlieder Scoring and performance
query id: Kindertotenlieder/The%20''Kindertotenlieder''%20and%20Mahler's%20life%20history, query text: Kindertotenlieder The ''Kindertotenlieder'' and Mahler's life history
query id: Civilisation%20(TV%20series)/Production, query text: Civilisation (TV series) Production
query id: Civilisation%20(TV%20series)/Reception, query text: Civilisation (TV series) Reception
query id: Civilisation%20(TV%20series)/Series%20outline, query text: Civilisation (TV series) Series outline
query id: Civilisation%20(TV%20series)/Proposed%20BBC%20Remake, query text: Civilisation (TV series) Proposed BBC Remake
query id: Northrop%20YB-35/Design%20and%20development, query text: Northrop YB-35 De

In [17]:
import ir_datasets, random, json, os
from collections import defaultdict
from tqdm import tqdm

# ── CONFIG ─────────────────────────────────────────────────────────────────────
DATASET     = "car/v1.5/train/fold0"
MAX_DOCS    = 10_000
NUM_QUERIES = 1_000
SEED        = 42
OUTPUT_DIR  = "subset"

random.seed(SEED)
ds = ir_datasets.load(DATASET)
docstore = ds.docs_store()

# ── 1) SAMPLE QUERIES & COLLECT QRELS ──────────────────────────────────────────
qrel_map = defaultdict(set)
for qr in ds.qrels_iter():
    if qr.relevance > 0:
        qrel_map[qr.query_id].add(qr.doc_id)

all_qids = list(qrel_map)
random.shuffle(all_qids)
selected_qids = set(all_qids[:NUM_QUERIES])

relevant_docs = set()
qrels_out   = []
for qid in selected_qids:
    for did in qrel_map[qid]:
        relevant_docs.add(did)
        qrels_out.append((qid, did, 1))

print(f"Selected {len(selected_qids)} queries with {len(relevant_docs)} relevant docs")

rel_texts = docstore.get_many(list(relevant_docs))

# ── 2) COLLECT RELEVANT DOCS ───────────────────────────────────────────────────
collected  = {did: rel_texts[did].text for did in relevant_docs}
needed_noise = MAX_DOCS - len(collected)
noise_count  = 0

for doc in ds.docs_iter():
    did = doc.doc_id
    if did in collected:
        continue            # skip any relevant doc we already have
    collected[did] = doc.text
    noise_count += 1
    if noise_count >= needed_noise:
        break

# ── 3) DUMP TO DISK ──────────────────────────────────────────────────────────────
os.makedirs(OUTPUT_DIR, exist_ok=True)
# docs.jsonl
with open(f"{OUTPUT_DIR}/docs.jsonl","w") as f:
    for did, text in collected.items():
        json.dump({"doc_id":did,"text":text}, f)
        f.write("\n")

# queries.jsonl
queries_map = {
    q.query_id: q.text
    for q in ds.queries_iter()
    if q.query_id in selected_qids
}
with open(f"{OUTPUT_DIR}/queries.jsonl","w") as f:
    for qid in selected_qids:
        json.dump({"query_id":qid,"text":queries_map[qid]}, f)
        f.write("\n")

# qrels.txt
with open(f"{OUTPUT_DIR}/qrels.txt","w") as f:
    for qid, did, rel in qrels_out:
        f.write(f"{qid} 0 {did} {rel}\n")

print("✅ Wrote subset to", OUTPUT_DIR)


[INFO] [starting] building docstore


Selected 1000 queries with 2399 relevant docs


docs_iter: 100%|████████████████| 29678367/29678367 [15:04<00:00, 32813.77doc/s]
[INFO] [finished] docs_iter: [15:04] [29678367doc] [32813.74doc/s]
[INFO] [finished] building docstore [15:04]


✅ Wrote subset to subset


In [4]:
import json
OUTPUT_DIR = "/home/guest/r12922050/GitHub/d2qplus/data/trec-car-10000"
docs = [json.loads(line) for line in open(f"{OUTPUT_DIR}/docs.jsonl")]
queries = [json.loads(line) for line in open(f"{OUTPUT_DIR}/queries.jsonl")]
qrels = [line.strip().split() for line in open(f"{OUTPUT_DIR}/qrels/test.trec")]
print("docs:", len(docs), "queries:", len(queries), "qrels:", len(qrels))
for doc in docs[:10]:
    print(doc)
for query in queries[:10]:
    print(query)
for qrel in qrels[:10]:
    print(qrel)

docs: 10000 queries: 1000 qrels: 2403
{'doc_id': 'e38f292eb9e669e1da6bd5aea8cc1b34992d5556', 'text': "Wharton served as a delegate to the Convention of 1832 from the District of Victoria. The convention had unanimously elected William Wharton to deliver the resolutions to the Coahuila y Tejas legislature in Saltillo and to the Mexican Congress in Mexico City. Following that convention's unsuccessful attempts to form a new state separate from Coahuila y Tejas (then a part of Mexico), he served as president of the follow-up Convention of 1833 and openly advocated complete independence from Mexico, in contrast to the moderate view held by native Texans and Stephen F. Austin. He later served as a delegate from the Columbia district to the Texas Consultation of 1835."}
{'doc_id': 'f3f70f4c3bf19b5673ae39917d923e4126d45582', 'text': 'The IDEA 2004 requires each school entity to publish a notice to parents, in newspapers or other media, including the student handbook and website regarding the 

In [ ]:
# change column 'doc_id' to '_id' in docs and queries
for doc in docs:
    doc['_id'] = doc.pop('doc_id')
for query in queries:
    query['_id'] = query.pop('query_id')

OUTPUT_DIR = "/home/guest/r12922050/GitHub/d2qplus/data/trec-car-10000"

# save docs and queries to json files
with open(f"{OUTPUT_DIR}/corpus_new.jsonl", "w") as f:
    for doc in docs:
        json.dump(doc, f)
        f.write("\n")
with open(f"{OUTPUT_DIR}/queries_new.jsonl", "w") as f:
    for query in queries:
        json.dump(query, f)
        f.write("\n")

In [7]:
import json
with open("/home/guest/r12922050/GitHub/d2qplus/gen/nfcorpus/Llama-3.2-1B-Instruct-GRPO-separate-reward.jsonl", "r") as f:
    obj = [json.loads(line) for line in f]
obj[0]

{'id': 'MED-10',
 'title': 'Statin Use and Breast Cancer Survival: A Nationwide Cohort Study from Finland',
 'text': 'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years)

In [10]:
with open(f"/home/guest/r12922050/GitHub/d2qplus/data/trec-car-10000/corpus.jsonl", "r") as f:
    corpus = [json.loads(line) for line in f]

# for every document in corpus, add 'predicted_queries' field with empty string
for doc in corpus:
    doc['id'] = doc.pop('_id')  # rename '_id' to 'id'
    doc['predicted_queries'] = ""
# save corpus to file
with open("/home/guest/r12922050/GitHub/d2qplus/gen/trec-car/text-only.jsonl", "w") as f:
    for doc in corpus:
        json.dump(doc, f)
        f.write("\n")
